In [7]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/srv/idp-radio-1'

In [8]:
import os 
import tensorflow as tf
from pathlib import Path

In [9]:
# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:42:00.0, compute capability: 7.5



In [10]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight

In [11]:
#CHEXPERT_BENCHMARKS, CHESTXRAY14_BENCHMARKS = generate_benchmarks ()

In [12]:
#CHEXPERT_BENCHMARKS.keys() 

In [13]:
#CHESTXRAY14_BENCHMARKS.keys()

In [14]:
#[m.name for m in METRICS]

In [15]:
#CHEXPERT_COLUMNS

In [16]:
!remote_access/get_tunnels.sh

URLs open for the following services:
[('tensorboard', 'http://f98d8522188f.ngrok.io'), ('jupyterlab', 'http://a22b25337789.ngrok.io')]


In [17]:
reduced_columns = ['Enlarged Cardiomediastinum',
                    'Cardiomegaly',
                    'Lung Opacity',
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture']
len(reduced_columns)

12

In [21]:
chexpert_benchmarks, _ = generate_benchmarks(path = Path(os.environ.get("CHEXPERT_FULL_PREPROCESSED_DATASET_DIRECTORY")),
                                                     name_suffix="_Masked",
                                                     classes=reduced_columns,
                                                     #train_labels = "nofinding_train.csv",
                                                     nan_replacement = -1, #float("NaN"),
                                                     batch_sizes = {"b": 32},
                                                     epoch_sizes = {"e": 2},
                                                     crop = {"C1": False},
                                                     split_seed = 6122156, )

Chestxray_BCE_E2_B32_C1_N12_Masked could not be created
Chestxray_WBCE_E2_B32_C1_N12_Masked could not be created
Chestxray_CWBCE_E2_B32_C1_N12_Masked could not be created


In [22]:
chexpert_benchmarks.keys()

dict_keys(['BCE_E2_B32_C1_N12_Masked', 'WBCE_E2_B32_C1_N12_Masked', 'CWBCE_E2_B32_C1_N12_Masked'])

In [24]:
chexpert_exp = simple_architecture_experiment(chexpert_benchmarks["WBCE_E13_B32_C1_N14"], DenseNet121, CHEXPERT_COLUMNS)

In [25]:
chexpert_exp.run()

Epoch 1/13
  94/4447 [..............................] - ETA: 13:48 - loss: 2.8796 - auc: 0.4977 - precision: 0.1596 - recall: 0.4930 - f2_score: 0.3478 - binary_accuracy: 0.4898 - learning rate: 1.0000e-04

KeyboardInterrupt: 

In [ ]:
mymodel = SimpleBaseArchitecture(DenseNet121, len(reduced_columns))

In [ ]:
optim = chexpert_benchmarks["CWBCE_E12_B32_C1_N12"].optimizer
optim

In [ ]:
loss = chexpert_benchmarks["CWBCE_E12_B32_C1_N12"].loss
loss

In [ ]:
metrics = chexpert_benchmarks["CWBCE_E12_B32_C1_N12"].metrics
METRICS = [tf.keras.metrics.AUC(multi_label=True, name="auc"),
           tf.keras.metrics.Precision(name="precision"),
           tf.keras.metrics.Recall(name="recall"),
           tf.keras.metrics.BinaryAccuracy(name="binary_accuracy")
           ]
METRICS

In [ ]:
mymodel.compile(optimizer=optim,
               loss=loss,
               metrics = metrics)

In [ ]:
mymodel(first_batch)

In [ ]:
train_result = mymodel.fit(x=traingen,
                           steps_per_epoch=len(traingen),
                           epochs=2,)

In [ ]:
traingen = chexpert_benchmarks["BCE_E12_B32_C1_N12"].traingen
first_batch, y_true = traingen[0]
y_true

In [ ]:
weights_pos , weights_neg = compute_class_weight(traingen)
weights_pos

In [ ]:
y_pred = mymodel.predict_on_batch(first_batch)
y_pred

In [ ]:
#for layer in mymodel.layers: print(layer.get_config(), layer.get_weights())

In [ ]:
predictions = experiment_inceptionnet_chexpert.model.predict(
            experiment_inceptionnet_chexpert.benchmark.testgen, steps=len(experiment_inceptionnet_chexpert.benchmark.testgen), verbose=1)

In [ ]:
true_labels = experiment_inceptionnet_chexpert.benchmark.testgen.get_encoded_labels()

In [ ]:
# model_guendel_chestxray14 = densenet(classes=len(CHESTXRAY14_COLUMNS))
model_guendel_chexpert = densenet(classes=len(CHEXPERT_COLUMNS))

#experiment_guendel_chestxray14 = Experiment(chestxray14_benchmark, model_guendel_chestxray14)
experiment_guendel_chexpert = Experiment(CHEXPERT_BENCHMARKS["WBCE_E10_B32"], model_guendel_chexpert, model_name="test_WBCE_32")



In [ ]:
#experiment_guendel_chestxray14_result =  experiment_guendel_chestxray14.run()

In [ ]:
experiment_guendel_chexpert_result =  experiment_guendel_chexpert.run()

In [ ]:
experiment_guendel_chexpert.evaluate()

In [ ]:
experiment_guendel_chexpert.save(upload=False)

In [ ]:
print("done")

In [ ]:
CHEXPERT_BENCHMARKS["WBCE_E10_B32"].as_dict()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.losses import Loss
from tensorflow.python.ops.losses import util as tf_losses_util
from tensorflow.python.framework import tensor_util
from tensorflow.python.framework import constant_op
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.keras import backend_config

In [ ]:
weights_pos = tf.ones((4)) * 20.0
weights_neg = tf.ones((4)) * 0.5
y_true = tf.constant([[0, -1, -1, 1], [-1, 0, -1, -1]], dtype=tf.float32)
y_pred = tf.constant([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]])
loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
loss

In [ ]:
y_true = tf.constant(y_true, dtype=tf.float32)
y_true

In [ ]:
y_pred = tf.constant(y_pred)
y_pred

In [ ]:
if tensor_util.is_tensor(y_pred) and tensor_util.is_tensor(y_true):
    y_pred, y_true = tf_losses_util.squeeze_or_expand_dimensions(
        y_pred, y_true)

epsilon_ = constant_op.constant(
     backend_config.epsilon(), dtype=y_pred.dtype.base_dtype)
y_pred = clip_ops.clip_by_value(y_pred, epsilon_, 1. - epsilon_)

In [ ]:
mask = tf.cast(tf.math.greater_equal(y_true, 0), tf.float32)

# Compute cross entropy from probabilities.
bce_pos = y_true * math_ops.log(y_pred + epsilon_)
bce_neg = (1 - y_true) * math_ops.log(1 - y_pred + epsilon_)

bce_pos = tf.math.multiply_no_nan(bce_pos, mask)
bce_neg = tf.math.multiply_no_nan(bce_neg, mask)

bce = bce_pos * weights_pos + \
    bce_neg * weights_neg

loss = tf.reduce_sum(-bce) / tf.reduce_sum(mask)
loss

In [ ]:
tf.reduce_sum(mask)

In [ ]:
bce_neg

In [ ]:
bce

In [ ]:

from tensorflow.keras import backend as K
